In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def dydx(x,y):
    #set the derivatives
    
    #our equations is d^2y/dx^2
    
    #so we can write 
    #dydx =z
    #dzdx=-y
    
    # We will set y = y[0]
    # We will set z = y[1]
    
    #declare an array
    y_derivs=np.zeros(2)
    
    #set dy/dx =z
    y_derivs[0] = y[1]
    
    #set dfdx = -y
    y_derivs[1] = -1.*y[0]
    
    return y_derivs


## Define the 4th order RK method

In [ ]:
#nv number of varbiable
#xi x and time i
#yi the values of y and z 
#h is a step
def rk4_mv_core(dydx,xi,yi,nv,h):
    #declare k? array
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
    
    
    #define x at 1/2 step
    x_ipoh = xi +0.5*h
        
    #define x at 1 step    
    x_ipo = xi + h

    y_temp = np.zeros(nv)
    
    #k1 Values
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    #k2 values
    y_temp[:] = yi[:] + (0.2*k1[:])
    x_temp = xi + (.2*h)
    y_derivs = dydx( x_temp , y_temp )
    k2[:] = h*y_derivs[:]
    
    #get k3 values
    y_temp[:] =yi[:] + (3/40)*k1[:] + ((9/40)*k2[:])
    x_temp = xi + ((3/10)*h)
    y_derivs = dydx(x_temp,y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    y_temp[:] = yi[:] + (3/10)*k1[:] + ((-9/10)*k2[:]) + ((6/5)*k3[:])
    x_temp = xi + ((3/5)*h)
    y_derivs = dydx( x_temp , y_temp )
    k4[:] = h * y_derivs[:]

    #get k5 values
    y_temp[:] = yi[:] + ((-11/54)*k1[:]) + ((5/2)*k2[:]) + ((-70/27)*k3[:]) + ((35/27)*k4[:])              
    x_temp = xi + (h)                
    y_derivs = dydx(x_temp, y_temp)         
    k5[:] = h * y_derivs[:]
    
                    
                    
    #get k6 values
    y_temp[:] = yi[:] + ((1631/55296)*k1[:]) + ((175/512)*k2[:]) + ((575/13824)*k3[:]) + ((44275/110592)*k4[:]) + ((253/4096)*k5[:])              
    x_temp = xi + ((7/8)*h)                              
    y_derivs = dydx(x_temp, y_temp)         
    k6[:] = h * y_derivs[:]               
      
                    
                    
                    
                    
    
    #advance y by a step h
    yipo = yi+ ((37/378)*k1)   + 0*k2 + (250/621)*k3 + ((125/594)*k4) + ((0)*k5) + ((512/1771)*k6)
                    
    yipo_star = yi + ((2825/27648)*k1) + 0*k2 + ((18575/48384)*k3) + ((13525/55296)*k4) +  ((277/14336)*k5) + ((.25)*k6)               
    
    Delta = np.fabs(yipo -  yipo_star)
                    
    return yipo,Delta

## Define an adaptive step size driver for RK4

In [ ]:
def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    #define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a max # of iterations
    imax = 10000
    
    #set a iteration variable
    i=0
    
    h_step = h
    
    yipo,Delta =  rk4_mv_core(dydx,x_i,y_i,nv,h_step)
    
    while(Delta.max()/tol > 1.0):
        h_step *=SAFETY * (Delta.max()/tol)**(-0.25)
            
        yipo,Delta = rk4_mv_core(dydx,x_i,y_i,nv,h_step)
        
        #check iterations
        if(i>imax):
            print("Too many iterations in rk4_mv_ad()")
            raise StopIteration("Ending after i = ", i)
            
        i +=1
        
    #next time, try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step * H_NEW_FAC)
    
    #return the answer
    return yipo,h_new,h_step

## Define a wrapper for RK4

In [ ]:
def rk4_mv(dydx,a,b,y_a,tol):
    
    #dydx is the deriative wrt x
    #a is the lower bound
    #b is the uper bond
    #y-a boundary Condution 
    
    xi = a
    yi = y_a.copy()
    
    #An initial step size == make very small
    h = 1.0e-4 * (b-a)
    
    #set a maximum number of iterations
    imax = 10000
    
    #set an iterations variable
    i =0
    
    #set the number of coupled odes to the size of y_a
    nv = len(y_a)  # y_a has the size of our boundery conditions
    
    x=np.full(1,a)
    y= np.full((1,nv),y_a)
    
    #set a flag
    flag = 1
    
    #loop till we reach the right side
    while(flag):
        
        #calcualte y_i+1
        yi_new, h_new,h_step = rk4_mv_ad(dydx, xi, yi,nv,h, tol)
        
        #Update the step
        h = h_new
        
        #Prevent an over step
        if(xi+h_step>b):
            
            #Take a smaller step
             h = b-xi
                
            #retake the step
             yi_new, h_new,h_step = rk4_mv_ad(dydx, xi, yi,nv,h, tol)   
           
             flag = 0
            
        #update the values
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the array
        x = np.append(x,xi)  #Add the answer into the new array
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:]=yi[:]
        del y
        y = y_new
        
        
        #prevent too mant iterations
        if(i>=imax):
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number =",i)
            
        i +=1
            
            #output some function
        s= "i = %3d\tx = %9.8f\th = %9.8f\tb =%9.8f" % (i,xi,h_step,b)
            #\tb print a tab (makes space) 
            #% print a float size of 8 digits behind the ecimal
        print (s)
            
            #break if new xi is == b
        if (xi==b):
            flag=0
        #return the answer
    return x,y
    


In [ ]:
a = 0.0
b = 2.0 * np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0

nv = 2

tolerance = 1.0e-6

#perform the integratoin 
x,y = rk4_mv(dydx,a,b,y_0,tolerance)

## Plot the result

In [ ]:
plt.plot(x,y[:,0],'o',label='y(x)')

plt.plot(x,y[:,1],'o',label='dydx(x)')

xx = np.linspace(0,2.0*np.pi,1000)

plt.plot(xx,np.sin(xx),label='sin(x)')

plt.plot(xx,np.cos(xx),label='cos(x)')

plt.xlabel('X')

plt.ylabel('y,dy/dx')
plt.legend(frameon=False)
plt.show()

In [ ]:
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0]-sine)

dydx_error = ( y[:,1]-cosine)

plt.plot(x,y_error,label='y(x) Error')

plt.plot(x, dydx_error,label = 'dydx(x)  Error')

plt.legend(frameon = False)
plt.show()